In [1]:
import os
from astroquery.sdss import SDSS
from astropy.table import Table
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import time
from astroquery.vizier import Vizier
from http.client import RemoteDisconnected
from urllib.error import ContentTooShortError

In [2]:
file_name = 'final_flux.csv'
if not os.path.exists(file_name):
    final_flux = np.asarray([])
    final_wavelength = np.asarray([])
else:
    final_flux = np.loadtxt(file_name,delimeter=',')

In [3]:
df_combined = pd.read_csv("df_combined.csv")

In [4]:
df_combined.shape

(45000, 5)

In [5]:
final_flux.shape

(0,)

In [ ]:
wavelength_check = []
no_spectra = []
for i in tqdm(range(final_flux.shape[0],final_flux.shape[0]+10)):
    if i >= df_combined.shape[0]:
        print("df_combined complete")
        break
    
    sp = SDSS.get_spectra(plate = df_combined['plate'][i],mjd=df_combined['mjd'][i],fiberID=df_combined['fiberid'][i], data_release=17, timeout=120, cache=False)

    if sp == None:
        fail = [df_combined['plate'][i], df_combined['mjd'][i], df_combined['fiberid'][i]]
        no_spectra.append(fail)
        del fail
        continue


    hdu_list = sp[0]
    data = hdu_list[1].data

    wavelength = 10**data['loglam']  # Convert from log wavelength to wavelength
    flux = data['flux']

    wavelength = np.asarray(wavelength[0:4000]).astype('float64')
    flux = np.asarray(flux[0:4000]).astype('float64')

    if len(final_wavelength)<len(wavelength):
        del final_wavelength
        final_wavelength = wavelength

    """
    if i == 0:
        small_wavelength = len(wavelength)
        final_wavelength = wavelength
        c = 0
    
    if i!=0 and small_wavelength<len(wavelength):
        final_wavelength = wavelength
        c = i
    """


    wavelength_check.append(wavelength)
    final_flux.append(flux)
    
    del sp
    del wavelength
    del flux
    del hdu_list
    del data


    

  0%|          | 0/10 [00:00<?, ?it/s]

In [ ]:
final_flux